In [1]:
import pandas as pd
import datetime as dt
import plotly.express as px
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../data/processed/emprestimos_exemplares.csv')

df.head()

In [2]:
df = pd.read_parquet('../data/processed/emprestimos_exemplares.parquet')

df.head()

,id_emprestimo,codigo_barras_emprestimo,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,codigo_barras_exemplar,colecao,biblioteca,status_material,localizacao,CDU
0,709684,L095049,None,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2008023265,ALUNO DE GRADUAÇÃO,13259,L095049,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,Ciências aplicadas
1,709685,L167050,None,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2008023265,ALUNO DE GRADUAÇÃO,70865,L167050,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,Ciências aplicadas
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2008112114,ALUNO DE PÓS-GRADUAÇÃO,195347,2006017618,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640,Ciências aplicadas
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,200721119,ALUNO DE GRADUAÇÃO,131639,L184117,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,609,Ciências aplicadas
4,709688,2007027500,2010/01/19 15:59:26.464000000,2010/01/04 08:27:33.569000000,2010/02/03 13:06:30.662000000,2009045971,ALUNO DE GRADUAÇÃO,225646,2007027500,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,686,Ciências aplicadas


In [ ]:
len(df.id_emprestimo.drop_duplicates())

In [ ]:
len(df)

In [5]:
df.dtypes

id_emprestimo                        int64
codigo_barras_emprestimo            object
data_renovacao                      object
data_emprestimo             datetime64[ns]
data_devolucao                      object
matricula_ou_siape                  object
tipo_vinculo_usuario                object
id_exemplar                          int64
codigo_barras_exemplar              object
colecao                             object
biblioteca                          object
status_material                     object
localizacao                          int64
CDU                               category
dtype: object

A diretoria da biblioteca gostaria de entender se a quantidade de empréstimos está diminuindo, aumentando ou permanecendo igual ao decorrer dos últimos anos.

In [ ]:
df['data_emprestimo'].value_counts()

In [4]:
# convertendo o tipo de data_emprestimo para datetime

df['data_emprestimo'] = pd.to_datetime(df['data_emprestimo'], dayfirst=True, format='ISO8601')

In [ ]:
# verifique qual é a quantidade total de exemplares emprestados por cada ano
# plote um gráfico de linhas.

qtd_por_ano = df['data_emprestimo'].value_counts().reset_index()

qtd_por_ano.rename(
    columns={
        'data_emprestimo': 'data',
        'count': 'quantidade'
    },
    inplace=True
)

qtd_por_ano


In [ ]:
# verifique qual é a quantidade total de exemplares emprestados por cada ano
# plote um gráfico de linhas.

qtd_por_ano_agrupado = qtd_por_ano.groupby(
    by=qtd_por_ano.data.dt.year).sum(numeric_only=True).reset_index()

qtd_por_ano_agrupado.rename(
    columns={
        'data': 'ano'
    },
    inplace=True
)

# qtd_por_ano_agrupado.columns

fig = px.line(
    qtd_por_ano_agrupado,
    x='ano',
    y='quantidade',
    markers=True,
    title='verifique qual é a quantidade total de exemplares emprestados por cada ano'
)

fig.show()

In [ ]:
qtd_por_ano_agrupado

In [ ]:
sum(qtd_por_ano_agrupado.quantidade)

In [ ]:
# gerenciar a programação de férias dos colaboradores de acordo com os meses de menor demanda;
# programar atividades que não sejam de atendimento ao usuário para períodos específicos de
# menor demanda.

# Portanto, gere uma tabela com a quantidade total de exemplares emprestados por mês e descubra
# quais meses são os que possuem a maior quantidade de empréstimos realizados.

qtd_por_mes_agrupado = qtd_por_ano.groupby(
    by=qtd_por_ano.data.dt.month
    ).sum(numeric_only=True).reset_index()

qtd_por_mes_agrupado.rename(
    columns={
        'data': 'mes'
    },
    inplace=True
)

# qtd_por_mes_agrupado

mes_renomear = {
    1: 'jan',
    2: 'fev',
    3: 'mar',
    4: 'abr',
    5: 'mai',
    6: 'jun',
    7: 'jul',
    8: 'ago',
    9: 'set',
    10: 'out',
    11: 'nov',
    12: 'dez',
}

qtd_por_mes_agrupado['mes'] = qtd_por_mes_agrupado['mes'].map(mes_renomear)

# qtd_por_mes_agrupado.sort_values(by='quantidade',ascending=False)

# Plote um gráfico de linhas.
fig = px.line(
    qtd_por_mes_agrupado,
    x='mes',
    y='quantidade',
    markers=True,
    title='Quantidade total de exemplares emprestados por mês',
)

fig.show()

In [ ]:
qtd_por_mes_agrupado

In [ ]:
qtd_por_ano.data.dt.hour.unique()

In [ ]:
# Além do gerenciamento anual das atividades, a diretoria também necessita que seja planejada 
# uma programação diária das atividades. Por este motivo, verifique quais foram os horários
# com maior quantidade de empréstimos ao longo de um dia inteiro.


qtd_por_hora_agrupado = qtd_por_ano.groupby(
    by=qtd_por_ano.data.dt.hour
    ).sum(numeric_only=True).reset_index()

qtd_por_hora_agrupado.rename(
    columns={
        'data': 'hora'
    },
    inplace=True
)

# qtd_por_hora_agrupado

# Plote um gráfico de barras e analise quais seriam os melhores horários para alocar as demais
# atividades que não sejam de atendimento ao usuário.
fig = px.bar(
    qtd_por_hora_agrupado,
    x='hora',
    y='quantidade',
    title='Quantidade total de exemplares emprestados por hora',
)

fig.show()

In [ ]:
# O objetivo será entender a quantidade de empréstimos a partir das variáveis categóricas do 
# seu conjunto de dados.
# Para explorar os dados, alguns questionamentos serão pertinentes para a diretoria das bibliotecas,
# como:
# Vamos explorar algumas das variáveis categóricas das quais precisaremos extrair mais informações. 
# Elas são:
#   Tipo de vínculo
#   Coleção
#   Biblioteca
#   Classificação geral da CDU

# Ao se analisar uma nova variável, é bem interessante verificar os valores únicos delas.

tipo_vinculo = df['tipo_vinculo_usuario'].unique()

colecao = df['colecao'].unique()

biblioteca = df['biblioteca'].unique()

cdu = df['CDU'].unique()

In [ ]:
tipo_vinculo

In [ ]:
colecao

In [ ]:
biblioteca

In [ ]:
cdu

In [ ]:
df.columns

In [ ]:
# Gere uma tabela de frequência e com o percentual para cada variável.
# Como é um trabalho repetitivo, crie uma função que gere a tabela com os valores.

def gerar_tabela_frequencia(campo):
    df_tabela = pd.DataFrame(df[campo].value_counts().reset_index())

    df_tabela.rename(
        columns={
            'count':'quantidade'
        },
        inplace=True
    )

    df_tabela['porcentagem'] = round((df_tabela['quantidade'] / df_tabela['quantidade'].sum())*100, 10)

    return df_tabela


In [ ]:
# “Como se distribuem os empréstimos de exemplares pelos tipos de vínculo dos usuários?”
# Desta forma, a diretoria poderá entender qual é o público que está utilizando a biblioteca 
# e assim tomar decisões em continuar com a estratégia de negócio atual ou modificá-la.

gerar_tabela_frequencia('tipo_vinculo_usuario')

In [ ]:
# Quais coleções são mais emprestadas?
# Da mesma forma, as coleções. Ranquear as coleções mais emprestadas pelo público, será bastante
# importante para a estratégia atual.

gerar_tabela_frequencia('colecao')

In [ ]:
# Quais são as bibliotecas com mais ou menos quantidade de empréstimos?
# Assim, a diretoria conseguirá entender onde ela deverá melhorar e focar suas iniciativas.

gerar_tabela_frequencia('biblioteca')

In [ ]:
# De quais temas da CDU tem os exemplares mais emprestados?

gerar_tabela_frequencia('CDU')

In [ ]:
# avaliar dentre os alunos de graduação e pós graduação a distribuição de empréstimos mensais 
# por ano realizados entre 2010 e 2022 da coleção que tiver a maior frequência de empréstimos.

# Distribuição mensal por alunos de graduação
alunos_graduacao = df[df['tipo_vinculo_usuario'] == 'ALUNO DE GRADUAÇÃO']

colecao = alunos_graduacao['colecao'].value_counts().idxmax()

emprestimos_graduacao = alunos_graduacao[alunos_graduacao['colecao'] == colecao].copy()

emprestimos_graduacao['ano'] = emprestimos_graduacao['data_emprestimo'].dt.year
emprestimos_graduacao['mes'] = emprestimos_graduacao['data_emprestimo'].dt.month

mes_renomear = {
    1: 'jan',
    2: 'fev',
    3: 'mar',
    4: 'abr',
    5: 'mai',
    6: 'jun',
    7: 'jul',
    8: 'ago',
    9: 'set',
    10: 'out',
    11: 'nov',
    12: 'dez',
}

emprestimos_graduacao['mes'] = emprestimos_graduacao['mes'].map(mes_renomear)

emprestimos_graduacao = emprestimos_graduacao.loc[:,['ano', 'mes']]
emprestimos_graduacao = emprestimos_graduacao.value_counts().to_frame('quantidade').reset_index()

emprestimos_graduacao['mes'].unique()

In [ ]:
meses = [
    'jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez'
]

fig = px.box(
    emprestimos_graduacao,
    x='mes',
    y='quantidade',
    color='mes',
    category_orders={
        'mes': meses
    },
    title='Graduação<br>Distribuição Mensal de Empréstimos dos anos 2010 a 2022 - Coleção Mais Emprestada'
)

fig.show()

In [ ]:
alunos_pos_graduacao = df[df['tipo_vinculo_usuario'] == 'ALUNO DE PÓS-GRADUAÇÃO']

colecao = alunos_pos_graduacao['colecao'].value_counts().idxmax()

emprestimos = alunos_pos_graduacao[alunos_pos_graduacao['colecao'] == colecao].copy()

emprestimos['ano'] = emprestimos['data_emprestimo'].dt.year
emprestimos['mes'] = emprestimos['data_emprestimo'].dt.month


emprestimos = emprestimos.loc[:,['ano', 'mes']]
emprestimos = emprestimos.value_counts().to_frame('quantidade').reset_index()

emprestimos

In [ ]:
fig = px.box(
    emprestimos,
    x='ano',
    y='quantidade',
    color='ano',
    category_orders={
        'ano': sorted(emprestimos_graduacao['ano'].unique())
    },
    title='Pós-Graduação - Distribuição Mensal de Empréstimos dos anos 2010 a 2022<br> Coleção Mais Emprestada'
)

fig.show()

In [ ]:
df_records = pd.read_csv('../data/processed/students_records/records.csv')

df_records.head()

In [13]:
df_records = pd.read_parquet('../data/processed/students_records/records.parquet')
df_records.head()

,matricula_ou_siape,tipo_vinculo_usuario,curso
0,190001688,ALUNO DE GRADUAÇÃO,MATEMÁTICA
1,199014489,ALUNO DE GRADUAÇÃO,CIÊNCIAS BIOLÓGICAS
2,199735674,ALUNO DE GRADUAÇÃO,MATEMÁTICA
3,199812431,ALUNO DE GRADUAÇÃO,LETRAS
4,199823670,ALUNO DE GRADUAÇÃO,ECOLOGIA


In [ ]:
graduacao = df.query("tipo_vinculo_usuario == 'ALUNO DE GRADUAÇÃO'")

graduacao

In [18]:
graduacao_pos_2015 = df[df['tipo_vinculo_usuario'] == 'ALUNO DE GRADUAÇÃO']

graduacao_pos_2015 = graduacao_pos_2015.loc[:,['matricula_ou_siape','data_emprestimo']]

# graduacao_pos_2015['data_emprestimo'] = pd.to_datetime(graduacao_pos_2015['data_emprestimo'])

graduacao_pos_2015 = graduacao_pos_2015[graduacao_pos_2015['data_emprestimo'].dt.year > 2015]

graduacao_pos_2015 = graduacao_pos_2015.reset_index(drop=True)

graduacao_pos_2015.head()

,matricula_ou_siape,data_emprestimo
0,2014035816,2016-01-02 08:01:14.113
1,2014036205,2016-01-02 10:12:01.462
2,2014036205,2016-01-02 10:12:01.529
3,2013042932,2016-01-02 10:21:42.118
4,2013042932,2016-01-02 10:21:42.162


In [10]:
graduacao.isna().sum()

matricula_ou_siape    0
data_emprestimo       0
dtype: int64

In [11]:
graduacao.isnull().sum()

matricula_ou_siape    0
data_emprestimo       0
dtype: int64

In [33]:
cursos_selecionados = df_records[df_records['curso'].isin(['BIBLIOTECONOMIA','CIÊNCIAS SOCIAIS','COMUNICAÇÃO SOCIAL','DIREITO','FILOSOFIA','PEDAGOGIA'])]
cursos_selecionados

,matricula_ou_siape,tipo_vinculo_usuario,curso
8,200031011,ALUNO DE GRADUAÇÃO,DIREITO
9,200045080,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
15,200115053,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
18,200123382,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
19,200123730,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
...,...,...,...
89263,20200097481,ALUNO DE GRADUAÇÃO,PEDAGOGIA
89270,20200107087,ALUNO DE GRADUAÇÃO,PEDAGOGIA
89271,20200012706,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
89274,20200009344,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA


In [ ]:
usuarios_cursos = graduacao_pos_2015.merge(cursos_selecionados)

usuarios_cursos

,matricula_ou_siape,data_emprestimo,tipo_vinculo_usuario,curso,ano_emprestimo
0,2013042932,2016-01-02 10:21:42.118,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS,2016
1,2013042932,2016-01-02 10:21:42.162,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS,2016
2,2012952924,2016-01-02 12:15:10.714,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS,2016
3,2012952924,2016-01-02 12:23:44.479,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS,2016
4,2012952924,2016-01-02 12:23:44.544,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS,2016
...,...,...,...,...,...
186287,20180065770,2022-12-30 10:12:11.036,ALUNO DE GRADUAÇÃO,DIREITO,2022
186288,20180065770,2022-12-30 10:12:11.036,ALUNO DE GRADUAÇÃO,DIREITO,2022
186289,20170122827,2022-12-30 13:51:17.326,ALUNO DE GRADUAÇÃO,DIREITO,2022
186290,20170122827,2022-12-30 13:51:17.446,ALUNO DE GRADUAÇÃO,DIREITO,2022


In [34]:
usuarios_cursos['ano_emprestimo'] = usuarios_cursos['data_emprestimo'].dt.year

In [ ]:
emprestimos_cursos = usuarios_cursos.groupby(['ano_emprestimo', 'curso']).size().reset_index(name='quantidade')
emprestimos_cursos

In [46]:
emprestimos_cursos_pivot = emprestimos_cursos.pivot_table(
    index='curso',
    columns='ano_emprestimo',
    values='quantidade',
    fill_value = '-',
    aggfunc= 'sum',
    margins = True,
    margins_name = 'TOTAL',
)

emprestimos_cursos_pivot

ano_emprestimo,2016,2017,2018,2019,2020,2021,2022,TOTAL
curso,,,,,,,,
BIBLIOTECONOMIA,7044,7307,6567,6320,964,69,1175,29446
CIÊNCIAS SOCIAIS,6984,6926,6680,6738,909,76,951,29264
COMUNICAÇÃO SOCIAL,7973,8253,6953,5885,807,66,942,30879
DIREITO,7363,7885,8466,9203,1352,99,1331,35699
FILOSOFIA,7624,7427,6814,6248,893,101,1136,30243
PEDAGOGIA,7174,7464,7094,6783,957,123,1166,30761
TOTAL,44162,45262,42574,41177,5882,534,6701,186292
